BATCH LEAST SQUARES ALGORITHM

In [1]:
import numpy as np

In [2]:
major_train = np.array([
    12250, 11500, 11250, 11000, 11960, 12140, 13000, 13800, 12950, 12600,
    11170, 12930, 13000, 11130, 11250, 12000, 12900, 11990, 12010, 12000,
    14490, 12900, 13220, 14000
])

minor_train = np.array([
    3750, 6500, 9000, 11000, 9000, 6510, 3750, 3750, 6500, 9000,
    11110, 13000, 12500, 9000, 6500, 3750, 3750, 6500, 9000, 12000,
    14450, 9000, 6500, 3750
])

sigma_train = np.array([
    3.92176e-2, 2.90297e-2, 2.51901e-2, 1.63300e-2, 2.50463e-2, 3.22360e-2,
    4.37127e-2, 4.91016e-2, 3.60650e-2, 2.80437e-2, 1.65160e-2, 2.02390e-2,
    2.36600e-2, 2.19333e-2, 2.77190e-2, 3.88243e-2, 4.28953e-2, 3.15040e-2,
    2.58113e-2, 1.95610e-2, 2.41490e-2, 2.97153e-2, 3.81633e-2, 5.05537e-2
])

major_test = np.array([
    12911.1, 11092.3, 13545.8, 13012.1, 12150, 12904, 14000,
    11406, 12100, 13109, 11017.6, 12105.9, 12700
])

minor_test = np.array([
    12927, 10966.4, 14487.1, 12963.8, 3750, 3744.8, 3770.4,
    6520.3, 6535.5, 6525.3, 9000.7, 8975.1, 900.0
])

sigma_test = np.array([
    2.0273e-2, 1.59737e-2, 2.40827e-2, 2.02157e-2, 3.8150e-2,
    4.28953e-2, 5.05537e-2, 2.83967e-2, 3.2120e-2, 3.69773e-2,
    2.11967e-2, 2.57443e-2, 2.8504e-2
])

In [3]:
X_train = np.column_stack([major_train, minor_train])
y_train = sigma_train

X_test = np.column_stack([major_test, minor_test])
y_test = sigma_test

In [4]:
def fuzzy_bls_algo(
    X_train: np.column_stack,
    y_train: np.array,
    X_test: np.column_stack,
    y_test: np.array,
    n_sets: int = 3
) -> np.array:
    """
    Calculates fuzzy model with BLS algorithm

    Args:
        X_train: Train input
        y_train: Train output
        X_test: Test input
        y_test: Test output

    Returns:
        b: consequent parameter
        train_pred: train prediction
        test_pred: test prediction
    """

    # Normalizing data
    x1_train_norm = (
        (X_train[:, 0] - np.mean(X_train[:, 0])) / np.std(X_train[:, 0])
    )
    x2_train_norm = (
        (X_train[:, 1] - np.mean(X_train[:, 1])) / np.std(X_train[:, 1])
    )
    x1_test_norm = (
        (X_test[:, 0] - np.mean(X_test[:, 0])) / np.std(X_test[:, 0])
    )
    x2_test_norm = (
        (X_test[:, 1] - np.mean(X_test[:, 1])) / np.std(X_test[:, 1])
    )

    # number of rules
    R = n_sets ** 2

    # defining the centers
    c1 = np.linspace(np.min(x1_train_norm), np.max(x1_train_norm), n_sets)
    c2 = np.linspace(np.min(x2_train_norm), np.max(x2_train_norm), n_sets)

    # Defining widths
    sigma1 = (np.max(x1_train_norm) - np.min(x1_train_norm)) / n_sets
    sigma2 = (np.max(x2_train_norm) - np.min(x2_train_norm)) / n_sets

    # Matrix
    def matrix(
        x1_norm: np.column_stack,
        x2_norm: np.column_stack
    ) -> np.array:
        """
        Generate the regression matrix for fuzzy model

        Args:
            x1_norm:
            x2_norm:

        Returns:
            Phi matrix
        """
        N = len(x1_norm)
        Phi = np.zeros((N, R))

        for i in range(N):
            w = list()
            for j in range(n_sets):
                for k in range(n_sets):

                    # membership functions
                    mu_1 = np.exp(-.5 * (x1_norm[i] - c1[j]) / sigma1)**2
                    mu_2 = np.exp(-.5 * (x2_norm[i] - c2[j]) / sigma2)**2

                    # rule weight
                    w.append(mu_1 * mu_2)
                    
            # sum the weights
            w_sum = sum(w)
            
            if w_sum == 0:
                w_sum = 1e-10   # avoid zero division
                
            Phi[i, :] = [w_i / w_sum for w_i in w]

        return Phi

    Phi_train = matrix(x1_train_norm, x2_train_norm)

    # bls = (ΦᵀΦ)⁻¹ Φᵀ y
    bls = np.linalg.pinv(Phi_train.T @ Phi_train) @ Phi_train.T @ y_train

    y_train_pred = Phi_train @ bls

    Phi_test = matrix(x1_test_norm, x2_test_norm)
    y_test_pred = Phi_test @ bls

    return bls, y_train_pred, y_test_pred, c1, c2, sigma1, sigma2

In [5]:
bls, y_train_pred, y_test_pred, c1, c2, sigma1, sigma2 = fuzzy_bls_algo(
    X_train, y_train, X_test, y_test, n_sets=3
)

In [6]:
# Errors

rmse_train = np.sqrt(np.mean((y_train - y_train_pred)**2))
rmse_test = np.sqrt(np.mean((y_test - y_test_pred)**2))

mae_train = np.mean(np.abs(y_train - y_train_pred))
mae_test = np.mean(np.abs(y_test - y_test_pred))

In [7]:
# Results

print("BLS Algo")
for i, bls in enumerate(bls):
    print(f"Rule {i+1}: b = {bls:.6f}") 

print("Training Errors")
print(f"RMSE: {rmse_train:.6f}")
print(f"MAE: {mae_train:.6f}")

print("Testing Errors")
print(f"RMSE: {rmse_test:.6f}")
print(f"MAE: {mae_test:.6f}")

BLS Algo
Rule 1: b = 0.000080
Rule 2: b = 0.000080
Rule 3: b = 0.000080
Rule 4: b = 0.001601
Rule 5: b = 0.001601
Rule 6: b = 0.001601
Rule 7: b = 0.032155
Rule 8: b = 0.032155
Rule 9: b = 0.032155
Training Errors
RMSE: 0.009560
MAE: 0.007994
Testing Errors
RMSE: 0.009776
MAE: 0.008324


RECURSIVE LEAST SQUARES ALGORITHM

GRADIENT METHOD

CLUSTERING METHOD

LEARNING FROM EXAMPLES

MODIFIED LEARNING FROM EXAMPLES